In [1]:
from distributed import Client
from time import sleep
import random
import math

In [2]:
cl = Client()

In [3]:
cl

<Client: scheduler="127.0.0.1:33358" processes=8 cores=8>

### Now you should have a scheduler here: http://localhost:8787/status

In [4]:
def nsleep(num_secs):
    sleep(num_secs)
    return num_secs

In [5]:
def get_rand_secs(max_int):
    return random.randint(0, max_int)

In [6]:
def do_some_math_with_errors(number):
    if number > 30:
        return math.log(number)
    elif number > 15:
        return round(number / 3.0)
    elif number >= 5:
        return math.floor(number / 1.5)
    elif number <= 2:
        return number / 0
    return number ** 2

In [7]:
def do_some_math(number):
    if number > 30:
        return math.log(number)
    elif number > 15:
        return round(number / 3.0)
    elif number >= 5:
        return math.floor(number / 1.5)
    elif number <= 2:
        return number 
    return number ** 2

In [8]:
random_secs = cl.map(get_rand_secs, range(200))

In [9]:
random_secs

[<Future: status: finished, type: int, key: get_rand_secs-15cf6cc7b778ccba5f338b9642b976f3>,
 <Future: status: finished, type: int, key: get_rand_secs-8b23bedfee459a48e084267f84f89e9a>,
 <Future: status: finished, type: int, key: get_rand_secs-f08e5e543792426d21f54ae491a8cd59>,
 <Future: status: finished, type: int, key: get_rand_secs-72830a7e4b8372fdda9caefec3a028b2>,
 <Future: status: finished, type: int, key: get_rand_secs-47f7b163368031ba794cba1fbad7ed83>,
 <Future: status: finished, type: int, key: get_rand_secs-f117ecf78ede714749400d254de49505>,
 <Future: status: finished, type: int, key: get_rand_secs-1f06e007fcf980654cb4219f6a8dbd00>,
 <Future: status: finished, type: int, key: get_rand_secs-1fba9bec5cf3604f73fbf8779660e86b>,
 <Future: status: finished, type: int, key: get_rand_secs-a8e290851ed758218b61926548a3b839>,
 <Future: status: finished, type: int, key: get_rand_secs-d300f2cdc876f1848452bdabdd3fe457>,
 <Future: status: finished, type: int, key: get_rand_secs-9bd58d33afc1

In [10]:
cl.gather(random_secs)[:10]

[0, 0, 0, 1, 0, 4, 1, 7, 5, 7]

In [11]:
random_math = cl.map(do_some_math_with_errors, random_secs)

In [12]:
excs = [(e.traceback(), e.exception()) for e in random_math if e.exception()]

In [13]:
excs[0]

(<traceback at 0x7f833c77c908>, ZeroDivisionError('division by zero'))

In [14]:
random_math = cl.map(do_some_math, random_secs)

In [15]:
random_sleeps = cl.map(nsleep, random_math)

In [16]:
random_sleeps

[<Future: status: finished, type: int, key: nsleep-e75087c08cff8ccbe4e10e68ff1c46d4>,
 <Future: status: finished, type: int, key: nsleep-7d6df8bc10b883eb8532a6d0b42b50fa>,
 <Future: status: finished, type: int, key: nsleep-40c054137d1c1ea2eaa5f2decbea6ad2>,
 <Future: status: pending, key: nsleep-c29551ff0a4ce9bc0fb56aa1edc99145>,
 <Future: status: pending, key: nsleep-2d53657530cd2b850246300b5d0b1a6b>,
 <Future: status: pending, key: nsleep-e70b3349affe14aeec1b34db52071f3a>,
 <Future: status: pending, key: nsleep-fec39bfdf1e8293c4880913768ca2768>,
 <Future: status: pending, key: nsleep-36177ae2cb506a154cec1bebdfd4c813>,
 <Future: status: pending, key: nsleep-a702fb96763ad9a65137bc5e964ff07f>,
 <Future: status: pending, key: nsleep-39ff81da710943e123a7272df228a571>,
 <Future: status: pending, key: nsleep-6adb1bbd32af4a31e2592f0bcb83b5ad>,
 <Future: status: pending, key: nsleep-e777e9b8832302be3bdcc2f1134cd17a>,
 <Future: status: pending, key: nsleep-bcd56b100f40100395a4d79add9b98d9>,
 <

In [17]:
sum_sleep = cl.submit(sum, random_sleeps)

In [18]:
sum_sleep

<Future: status: pending, key: sum-dc03cb9f2ab049c238c983d24c2a01f0>

In [ ]:
sum_sleep.result()